### PREPARACION PARA ADOBE

Este documento busca la data , la relaciona y la sirve de acuerdo a las tablas y tipos de datos que solicitó el partner

In [1]:
import pandas as pd
from pymongo import MongoClient
from datetime import timedelta  
import csv


In [2]:
import datetime 
import pymongo

In [3]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb2 = myclient["ecdf"]
mycol_users = mydb2["users"]


In [4]:
mycol_clientes = mydb2["clientes"]
def users_to_dataframes(iterator, chunk_size: int):
  """Turn an iterator into multiple small pandas.DataFrame
  This is a balance between memory and efficiency
  """
  records = []
  frames = []
  for i, record in enumerate(iterator):
    records.append(record)
    if i % chunk_size == chunk_size - 1:
      frames.append(pd.DataFrame(records))
      records = []
  if records:
    frames.append(pd.DataFrame(records))
  return pd.concat(frames) if frames else pd.DataFrame()
  
users = users_to_dataframes(mycol_users.find({},{'_id':1,'email':1,'cliente_id':1,'createdAt':1}), 10000)


Excluir los usuarios anteriores al 2017 

In [5]:
users['createdAt'] = pd.to_datetime(users['createdAt'])

In [6]:
users=users.loc[users['createdAt'] > '2017-01-01 00:00:00.904716']

#### LoginSummary TABLE

In [7]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287395 entries, 7421 to 4818
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   _id         287395 non-null  object        
 1   createdAt   287395 non-null  datetime64[ns]
 2   cliente_id  287335 non-null  object        
 3   email       287395 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 11.0+ MB


In [8]:
users=users.rename(columns={ "_id": "user_id"})

#### Eliminando los user_id nulos

In [9]:
users.drop(users[users.email.isnull()].index, inplace = True)

In [10]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287395 entries, 7421 to 4818
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     287395 non-null  object        
 1   createdAt   287395 non-null  datetime64[ns]
 2   cliente_id  287335 non-null  object        
 3   email       287395 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 11.0+ MB


 AHORA BUSCAMOS LAS CONEXIONES

In [11]:
mydb2 = myclient["ecdf"]
mycol_conexion = mydb2["Conexion"]

In [12]:
def conexiones_to_dataframes(iterator, chunk_size: int):
  """Turn an iterator into multiple small pandas.DataFrame
  This is a balance between memory and efficiency
  """
  records = []
  frames = []
  for i, record in enumerate(iterator):
    records.append(record)
    if i % chunk_size == chunk_size - 1:
      frames.append(pd.DataFrame(records))
      records = []
  if records:
    frames.append(pd.DataFrame(records))
  return pd.concat(frames) if frames else pd.DataFrame()
  
conexiones = conexiones_to_dataframes(mycol_conexion.find({},{'_user':1,'email':1,'createdAt':1}), 10000)

print(conexiones.count())

_id          4107119
_user        4107119
createdAt    4107119
email        2091686
dtype: int64


In [13]:
conexiones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4107119 entries, 0 to 7118
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   _id        object
 1   _user      object
 2   createdAt  object
 3   email      object
dtypes: object(4)
memory usage: 156.7+ MB


In [14]:
conexiones=conexiones.rename(columns={'createdAt':'fecha_conexion','_user':'user_id'})

In [15]:
mydb3 = myclient["apiUser"]
mycol_usersautomaticos = mydb3["usersautomaticos"]

In [16]:
def usersautomaticos_to_dataframes(iterator, chunk_size: int):
  """Turn an iterator into multiple small pandas.DataFrame
  This is a balance between memory and efficiency
  """
  records = []
  frames = []
  for i, record in enumerate(iterator):
    records.append(record)
    if i % chunk_size == chunk_size - 1:
      frames.append(pd.DataFrame(records))
      records = []
  if records:
    frames.append(pd.DataFrame(records))
  return pd.concat(frames) if frames else pd.DataFrame()
  
usersautomaticos =usersautomaticos_to_dataframes(mycol_usersautomaticos.find({},{'_user':1,'createdAt':1}), 10000)


In [17]:
usersautomaticos=usersautomaticos.rename(columns={'createdAt':'fecha_conexion','_user':'user_id'})

In [18]:
mydb3 = myclient["logins"]
mycol_conexLog = mydb3["conexiones"]

In [19]:
def mycol_conexLog_to_dataframes(iterator, chunk_size: int):
  """Turn an iterator into multiple small pandas.DataFrame
  This is a balance between memory and efficiency
  """
  records = []
  frames = []
  for i, record in enumerate(iterator):
    records.append(record)
    if i % chunk_size == chunk_size - 1:
      frames.append(pd.DataFrame(records))
      records = []
  if records:
    frames.append(pd.DataFrame(records))
  return pd.concat(frames) if frames else pd.DataFrame()
  
conexLog =mycol_conexLog_to_dataframes(mycol_conexLog.find({},{'user_id':1,'fecha_conexion':1}), 10000)



### Unifico los accesos

In [20]:
accesos=pd.concat([conexLog, usersautomaticos,conexiones], axis=0)

In [21]:
accesos=accesos.drop([ 'email'],axis=1)

In [22]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287395 entries, 7421 to 4818
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     287395 non-null  object        
 1   createdAt   287395 non-null  datetime64[ns]
 2   cliente_id  287335 non-null  object        
 3   email       287395 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 11.0+ MB


In [23]:
users=users.drop([ 'email','cliente_id'],axis=1)

In [24]:
user_conexiones= users.merge(accesos, on=['user_id'], how="outer", indicator=True)

### A PARTIR DE AQUÍ SE TIENE QUE IR HOMOLOGANDO EL CÓDIGO

In [25]:
accesConsolidado=user_conexiones.drop([ 'createdAt','_id','_merge'],axis=1)

In [26]:
accesConsolidado['fecha_conexion'] = pd.to_datetime(accesConsolidado['fecha_conexion'])

In [27]:
accesConsolidadoLife=accesConsolidado.groupby('user_id').agg(conexioneslife=('fecha_conexion','count'))

###  Ahora hay que segmentar por fecha para ir llenando la tabla

In [28]:
ultimos30dias=pd.to_datetime((datetime.datetime.today() - timedelta(days=30)))

In [29]:
ultimos30dias=ultimos30dias.tz_localize('UTC')

In [30]:
mes_hoy=datetime.datetime.today().month

In [31]:
mes_ant=mes_hoy-1

In [32]:
primer_diamesAnterior=pd.to_datetime(datetime.datetime.today().replace(month=mes_ant,day=1,hour=00,minute=0,second=0,microsecond=0))

In [33]:
primer_diamesAnterior=primer_diamesAnterior.tz_localize('UTC')

In [34]:
ultimo_diamesAnterir=pd.to_datetime(datetime.datetime.today().replace(day=1,hour=00,minute=0,second=0,microsecond=0))

In [35]:
ultimo_diamesAnterir=ultimo_diamesAnterir.tz_localize('UTC')

In [36]:
user_conexionesLast30Days=accesConsolidado.loc[accesConsolidado['fecha_conexion'] > ultimos30dias]

In [37]:
user_conexionesLast30Days=user_conexionesLast30Days.groupby('user_id').agg(conexionesLast30Days=('fecha_conexion','count'))

#### Obtener el último login

In [38]:
cusLastLoginDate=accesConsolidado.groupby('user_id').agg(cusLastLoginDate=('fecha_conexion','last'))

#### Obtener cusLoginsThisMonth

In [39]:
hoy=datetime.datetime.today()

In [40]:
primer_diames=pd.to_datetime(datetime.datetime.today().replace(day=1,hour=00,minute=0,second=0,microsecond=0))

In [41]:
primer_diames=primer_diames.tz_localize('UTC')

In [42]:
user_cusLoginsThisMonth=accesConsolidado.loc[accesConsolidado['fecha_conexion'] > primer_diames]

In [43]:
user_cusLoginsThisMonth=user_cusLoginsThisMonth.groupby('user_id').agg(cusLoginsThisMonth=('fecha_conexion', 'count') )

#### Obtener cusLoginsPreviousMonth

In [44]:
user_conexiones_cusLoginsPreviousMonth=accesConsolidado.loc[(accesConsolidado['fecha_conexion'] > primer_diamesAnterior)&(accesConsolidado['fecha_conexion'] < ultimo_diamesAnterir)]

In [45]:
user_conexiones_cusLoginsPreviousMonth=user_conexiones_cusLoginsPreviousMonth.groupby('user_id').agg(cusLoginsPreviousMonth=('fecha_conexion','count') )

#### Obtener cusLoginsLast3Months 

In [46]:
def periodo3():
    hoy=datetime.datetime.today()
    mes_hoy=hoy.month
    mes_last3Month=mes_hoy-3
    if (mes_last3Month == 0):
        mes_last3Month=12
        ano=hoy.year-1
        limite_izq=pd.to_datetime(datetime.datetime.today().replace(year=ano,month=mes_last3Month,day=1,hour=00,minute=0,second=0,microsecond=0))
        limite_izq=limite_izq.tz_localize('UTC')
        print(limite_izq)
    if (mes_last3Month == -1):
        mes_last3Month=11
        ano=hoy.year-1
        limite_izq=pd.to_datetime(datetime.datetime.today().replace(year=ano,month=mes_last3Month,day=1,hour=00,minute=0,second=0,microsecond=0))
    if (mes_last3Month == -2):
        mes_last3Month=10
        ano=hoy.year-1
        limite_izq=pd.to_datetime(datetime.datetime.today().replace(year=ano,month=mes_last3Month,day=1,hour=00,minute=0,second=0,microsecond=0))
    else:
        ano=hoy.year
        limite_izq=pd.to_datetime(datetime.datetime.today().replace(year=ano,month=mes_last3Month,day=1,hour=00,minute=0,second=0,microsecond=0))
        limite_izq=limite_izq.tz_localize('UTC')
        return limite_izq

In [47]:
mes_inici3=periodo3()

In [48]:
user_conexiones_cusLoginsLast3Months=accesConsolidado.loc[(accesConsolidado['fecha_conexion'] > mes_inici3)&(accesConsolidado['fecha_conexion'] < ultimo_diamesAnterir)]

In [49]:
user_conexiones_cusLoginsLast3Months=user_conexiones_cusLoginsLast3Months.groupby('user_id').agg(cusLoginsLast3Months=('fecha_conexion', 'count') )

#### Obtener cusLoginsLast12Months

In [50]:
def periodo12():
    hoy=datetime.datetime.today()
    month=hoy.month
    ano=hoy.year-1
    limite_izq=pd.to_datetime(datetime.datetime.today().replace(year=ano,month=month,day=1,hour=00,minute=0,second=0,microsecond=0))
    limite_izq=limite_izq.tz_localize('UTC')

    return limite_izq

In [51]:
mes_inici12=periodo12()

In [52]:
user_conexiones_cusLoginsLast12Months=accesConsolidado.loc[(accesConsolidado['fecha_conexion']>mes_inici12)&(accesConsolidado['fecha_conexion'] < ultimo_diamesAnterir)]

In [53]:
user_conexiones_cusLoginsLast12Months=user_conexiones_cusLoginsLast12Months.groupby('user_id').agg(cusLoginsLast12Months=('fecha_conexion','count'))

In [54]:
user_conexiones_cusLoginsBefore12Month=accesConsolidado.loc[(accesConsolidado['fecha_conexion']<mes_inici12)&(accesConsolidado['fecha_conexion'] < ultimo_diamesAnterir)]

In [55]:
user_conexiones_cusLoginsBefore12Month=user_conexiones_cusLoginsBefore12Month.groupby('user_id').agg(LoginsBefore12Month=('fecha_conexion','count'))

## ARMANDO LA TABLA FINAL

In [56]:
frames = [cusLastLoginDate, user_cusLoginsThisMonth, user_conexionesLast30Days,user_conexiones_cusLoginsPreviousMonth,user_conexiones_cusLoginsLast3Months,user_conexiones_cusLoginsLast12Months,accesConsolidadoLife]

In [57]:
CDF_LoginSummary = pd.concat(frames)

In [58]:
table=pd.concat([cusLastLoginDate,user_cusLoginsThisMonth, user_conexionesLast30Days,user_conexiones_cusLoginsPreviousMonth,user_conexiones_cusLoginsLast3Months,user_conexiones_cusLoginsLast12Months,accesConsolidadoLife], axis=1, sort=False)

In [59]:
table=table.rename(columns={'cusLastLoginDate':'LAST_LOGIN_DATE','cusLastLoginDate':'LAST_LOGIN_DATE','cusLoginsLifetime':'LOGINS_LIFETIME','cusLoginsThisMonth':'LOGINS_THIS_MONTH','conexionesLast30Days':'LOGINS_LAST_30_DAYS','cusLoginsPreviousMonth':'LOGINS_PREVIOUS_MONTH','cusLoginsLast3Months':'LOGINS_LAST_3_MONTHS','cusLoginsLast12Months':'LOGINS_LAST_12_MONTHS','conexioneslife':'LOGINS_LIFETIME'})

In [60]:
table['CUSTOMER_ID'] = table.index

In [61]:
table = table[['CUSTOMER_ID','LAST_LOGIN_DATE','LOGINS_THIS_MONTH','LOGINS_LAST_30_DAYS','LOGINS_PREVIOUS_MONTH','LOGINS_LAST_3_MONTHS','LOGINS_LAST_12_MONTHS','LOGINS_LIFETIME']]

 #### Ahora vamos a determinar el nombre y el path

In [62]:
name='cdf_loginSummary_'+datetime.datetime.today().strftime('%Y%m%d_%H%M%S')

In [63]:
path='/home/tecnoboy/'+name+'.csv'

In [64]:
table['CUSTOMER_ID']=table['CUSTOMER_ID'].apply(str)

In [65]:
table['CUSTOMER_ID'] =table['CUSTOMER_ID']

In [66]:
table['LAST_LOGIN_DATE']=table['LAST_LOGIN_DATE'].apply(str)

In [67]:
table['LAST_LOGIN_DATE']=table['LAST_LOGIN_DATE'].apply(str)

In [68]:
table['LAST_LOGIN_DATE']=table['LAST_LOGIN_DATE'].map(lambda x: str(x)[:19])

In [69]:
table['LOGINS_THIS_MONTH']=table['LOGINS_THIS_MONTH'].fillna(0)

In [70]:
table['LOGINS_LAST_3_MONTHS']=table['LOGINS_LAST_3_MONTHS'].fillna(0)

In [71]:
table['LOGINS_LAST_30_DAYS']=table['LOGINS_LAST_30_DAYS'].fillna(0)

In [72]:
table['LOGINS_PREVIOUS_MONTH']=table['LOGINS_PREVIOUS_MONTH'].fillna(0)

In [73]:
table['LOGINS_LAST_3_MONTHS']=table['LOGINS_LAST_3_MONTHS'].fillna(0)

In [74]:
table['LOGINS_LAST_12_MONTHS']=table['LOGINS_LAST_12_MONTHS'].fillna(0)

In [75]:
table['LOGINS_LIFETIME']=table['LOGINS_LIFETIME'].fillna(0)

In [76]:
table['LOGINS_THIS_MONTH']=table['LOGINS_THIS_MONTH'].astype(int)

In [77]:
table['LOGINS_LAST_3_MONTHS']=table['LOGINS_LAST_3_MONTHS'].astype(int)

In [78]:
table['LOGINS_LAST_30_DAYS']=table['LOGINS_LAST_30_DAYS'].astype(int)

In [79]:
table['LOGINS_PREVIOUS_MONTH']=table['LOGINS_PREVIOUS_MONTH'].astype(int)

In [80]:
table['LOGINS_LAST_3_MONTHS']=table['LOGINS_LAST_3_MONTHS'].astype(int)

In [81]:
table['LOGINS_LAST_12_MONTHS']=table['LOGINS_LAST_12_MONTHS'].astype(int)

In [82]:
table['LOGINS_LIFETIME']=table['LOGINS_LIFETIME'].astype(int)

In [83]:
table['LOGINS_LIFETIME']=table['LOGINS_LIFETIME'].fillna('')

In [84]:
table['LAST_LOGIN_DATE']=table['LAST_LOGIN_DATE'].replace({'NaT' : ''}, regex=True)

In [85]:
table.to_csv(path, index=False, quoting=csv.QUOTE_NONNUMERIC)